In [1]:
import time
import os
from subprocess import Popen, PIPE, STDOUT
from datasets import (
    get_dataset_config_names,
    load_dataset)
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    pipeline)
import torch

In [3]:
es_server = Popen(
    args=["C:\\elasticsearch-8.2.0\\bin\\elasticsearch.bat"],
    stdout=PIPE,
    stderr=STDOUT)

In [11]:
for line in es_server.stdout:
    print(line)

b'warning: ignoring JAVA_HOME=C:\\Program Files\\Java\\jdk-14.0.1; using bundled JDK \r\n'
b'warning: ignoring JAVA_HOME=C:\\Program Files\\Java\\jdk-14.0.1; using ES_JAVA_HOME \r\n'
b'[2022-05-22T10:29:57,895][INFO ][o.e.n.Node               ] [DESKTOP-FCQS5HT] version[8.2.0], pid[7208], build[default/zip/b174af62e8dd9f4ac4d25875e9381ffe2b9282c5/2022-04-20T10:35:10.180408517Z], OS[Windows 11/10.0/amd64], JVM[Eclipse Adoptium/OpenJDK 64-Bit Server VM/18/18+36]\r\n'
b'[2022-05-22T10:29:57,917][INFO ][o.e.n.Node               ] [DESKTOP-FCQS5HT] JVM home [C:\\elasticsearch-8.2.0\\jdk], using bundled JDK [true]\r\n'
b'[2022-05-22T10:29:57,918][INFO ][o.e.n.Node               ] [DESKTOP-FCQS5HT] JVM arguments [-Des.networkaddress.cache.ttl=60, -Des.networkaddress.cache.negative.ttl=10, -Djava.security.manager=allow, -XX:+AlwaysPreTouch, -Xss1m, -Djava.awt.headless=true, -Dfile.encoding=UTF-8, -Djna.nosys=true, -XX:-OmitStackTraceInFastThrow, -XX:+ShowCodeDetailsInExceptionMessages, -Dio.ne

In [6]:
!curl --insecure -X GET "localhost:9200/?pretty"
#!curl --insecure -u elastic https://localhost:9200

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 9200 after 2247 ms: Connection refused


In [5]:
es_server.terminate()

In [2]:
model_ckpt = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

domains = get_dataset_config_names('subjqa')

subjqa = load_dataset("subjqa",name="books")

Reusing dataset subjqa (C:\Users\grego\.cache\huggingface\datasets\subjqa\books\1.1.0\e5588f9298ff2d70686a00cc377e4bdccf4e32287459e3c6baf2dc5ab57fe7fd)
100%|██████████| 3/3 [00:00<00:00, 142.59it/s]


In [4]:
dfs = {split: dset.to_pandas() for split, dset 
    in subjqa.flatten().items()}

for split, df in dfs.items():
    print(f"Number of questions in {split}: {df['id'].nunique()}")

qa_cols = ['title', 'question', 'answers.text',
    'answers.answer_start', 'context']

sample_df = dfs["train"][qa_cols].sample(10, random_state=7)

sample_df

Number of questions in train: 1314
Number of questions in test: 345
Number of questions in validation: 256


,title,question,answers.text,answers.answer_start,context
463,0312576463,Why none is likeable?,"[The characters were not likable, and it often...","[119, 156]",It's the classic &#34;disconnected from parent...
284,0307265439,What is the sentiment of the story?,[],[],I rate books on how much of an impact they hav...
34,0007442920,What book is ?,[this book though luckily there is no love tri...,"[350, 350]",Insurgent takes off where Divergent ended. We ...
263,0143170090,Is the movie dialogue good?,[],[],"I found the story interesting, and yes, distur..."
151,0099464462,What is the main concept of books?,[],[],This book has elicited in me mixed emotions. T...
877,0439023513,How is the book?,[am floored at how bad this book],[86],(Spoiler alert)I just finished reading Mocking...
1159,1476730091,Do you study chemistry or electic?,[],[],Finally! I have been going through romances in...
737,0399159347,Do you consider the author a person skilled?,[that one can stand],[208],"The writer is very good at her trade, but the ..."
830,0439023483,What about action ?,[the action was great],[171],"Great book, to begin the Hunger Games series. ..."
268,0143170104,Is it really only one?,[],[],This story starts about a year and half after ...


In [5]:
pipe = pipeline("question-answering",model=model, tokenizer=tokenizer)

In [6]:
start_time = time.time()
train_res = pipe(
    question=sample_df["question"].tolist(),
    context=sample_df["context"].tolist())
elapsed_time = time.time() - start_time
print(f"{round(elapsed_time)} seconds")

C:\Users\grego\Anaconda3\envs\py39\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


3 seconds


In [7]:
train_res

[{'score': 0.00019147747661918402,
  'start': 123,
  'end': 150,
  'answer': 'characters were not likable'},
 {'score': 0.7689992189407349,
  'start': 1156,
  'end': 1166,
  'answer': 'depressing'},
 {'score': 0.8209125995635986, 'start': 0, 'end': 9, 'answer': 'Insurgent'},
 {'score': 0.09765518456697464,
  'start': 1196,
  'end': 1274,
  'answer': 'I would think the sexual lives of the main characters would be more disturbing'},
 {'score': 0.17623278498649597,
  'start': 189,
  'end': 202,
  'answer': 'romance novel'},
 {'score': 0.23674839735031128,
  'start': 1345,
  'end': 1360,
  'answer': 'flat out brutal'},
 {'score': 0.888448178768158,
  'start': 2085,
  'end': 2100,
  'answer': 'Side Characters'},
 {'score': 0.42023810744285583,
  'start': 0,
  'end': 36,
  'answer': 'The writer is very good at her trade'},
 {'score': 0.4013780951499939, 'start': 186, 'end': 191, 'answer': 'great'},
 {'score': 0.04846242070198059,
  'start': 931,
  'end': 982,
  'answer': 'This book dragged o

In [1]:
import os
os.getcwd()

'c:\\Users\\grego\\Documents\\GitHub\\hfmodels\\hfmodels'